# <span style="color:blue">MBA em Ciência de Dados</span>
# <span style="color:blue">Análise de Dados com Base em Processamento Massivo em Paralelo</span>

## <span style="color:blue">Avaliação Final - Notebook </span>

**Material Produzido por:**<br>
>**Profa. Dra. Cristina Dutra de Aguiar**<br>


**CEMEAI - ICMC/USP São Carlos**

Este *notebook* deve conter as respostas para as consultas analíticas solicitadas nas Questões 7, 8, 9 e 10. É possível especificar as consultas analíticas usando Pandas, o método spark.sql() ou os métodos do módulo pyspark.sql. As consultas devem ser especificadas na seção 6, na célula indicada para resposta.

**IMPORTANTE**

- **As respostas para as Questões 7, 8, 9 e 10 somente serão consideradas se forem especificadas no *notebook*. Portanto, independentemente da alternativa estar certa ou errada, se a consulta analítica correspondente não for especificada, a alternativa será considerada errada.**

- **Caso uma alternativa esteja certa, porém a especificação da consulta analítica correspondente estiver errada no *notebook*, a alternativa será considerada errada.**


O *notebook* contém a constelação de fatos da BI Solutions que deve ser utilizada para responder às questões e também todas as bibliotecas, bases de dados, inicializações, instalações, importações, geração de dataFrames, geração de visões temporárias e conversão dos tipos de dados necessárias para a realização da questão. Portanto, o *notebook* está preparado para ser executado usando Pandas, o método spark.sql() e os métodos do módulo pyspark.sql.

O uso do *framework* Spark requer diversas configurações no ambiente de desenvolvimento para executar o *notebook*. Dado que tal complexidade foge do escopo de nossa disciplina, recomenda-se que o *notebook* seja executado na plataforma de desenvolvimento COLAB. O uso do COLAB  proporciona um ambiente de desenvolvimento pré-configurado e remove a complexidade de instalação e configuração de pacotes e *frameworks* que são utilizados na disciplina.

**INSTRUÇÕES DE ENTREGA**

**O que deve ser entregue:**
- **O notebook com as respostas no formato .ipynb**
- **O notebook com as respostas no formato .pdf**

**Ambos arquivos devem ser nomeados usando o primeiro nome e o último sobrenome do aluno. Por exemplo: CristinaAguiar.ipynb e CristinaAguiar.pdf.**

Boa avaliação!

#1 Constelação de Fatos da BI Solutions

A aplicação de *data warehousing* da BI Solutions utiliza como base uma constelação de fatos, conforme descrita a seguir.

**Tabelas de dimensão**

- data (dataPK, dataCompleta, dataDia, dataMes, dataBimestre, dataTrimestre, dataSemestre, dataAno)
- funcionario (funcPK, funcMatricula, funcNome, funcSexo, funcDataNascimento, funcDiaNascimento, funcMesNascimento, funcAnoNascimento, funcCidade, funcEstadoNome, funcEstadoSigla, funcRegiaoNome, funcRegiaoSigla, funcPaisNome, funcPaisSigla)
- equipe (equipePK, equipeNome, filialNome, filialCidade, filialEstadoNome, filialEstadoSigla, filialRegiaoNome, filialRegiaoSigla, filialPaisNome, filialPaisSigla)
- cargo (cargoPK, cargoNome, cargoRegimeTrabalho, cargoEscolaridadeMinima, cargoNivel)
- cliente (clientePK, clienteNomeFantasia, clienteSetor, clienteCidade, clienteEstadoNome, clienteEstadoSigla, clienteRegiaoNome, clienteRegiaoSigla, clientePaisNome, clientePaisSigla)

**Tabelas de fatos**
- pagamento (dataPK, funcPK, equipePK, cargoPK, salario, quantidadeLancamentos)
- negociacao (dataPK, equipePK, clientePK, receita, quantidadeNegociacoes)


#2 Obtenção dos Dados da BI Solutions


## 2.1 Baixando o Módulo wget

Para baixar os dados referentes ao esquema relacional da constelação de fatos da BI Solutions, é utilizado o módulo  **wget**. O comando a seguir realiza a instalação desse módulo. <br>

In [0]:
#instalando o módulo wget
%%capture
!pip install -q wget
!mkdir data

## 2.2 Obtenção dos Dados das Tabelas de Dimensão

Os comandos a seguir baixam os dados que povoam as tabelas de dimensão.

In [0]:
#baixando os dados das tabelas de dimensão
import wget

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/data.csv"
wget.download(url, "data/data.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/funcionario.csv"
wget.download(url, "data/funcionario.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/equipe.csv"
wget.download(url, "data/equipe.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cargo.csv"
wget.download(url, "data/cargo.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cliente.csv"
wget.download(url, "data/cliente.csv")

'data/cliente.csv'

## 2.3 Obtenção dos Dados Tabelas de Fatos

Os comandos a seguir baixam os dados que povoam as tabelas de fatos.

In [0]:
#baixando os dados das tabelas de fatos
url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/pagamento.csv"
wget.download(url, "data/pagamento.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/negociacao.csv"
wget.download(url, "data/negociacao.csv")

'data/negociacao.csv'

# 3 Apache Spark Cluster

## 3.1 Instalação

Neste *notebook* é criado um *cluster* Spark composto apenas por um **nó mestre**. Ou seja, o *cluster* não possui um ou mais **nós de trabalho** e o **gerenciador de cluster**. Nessa configuração, as tarefas (*tasks*) são realizadas no próprio *driver* localizado no **nó mestre**.

Para que o cluster possa ser criado, primeiramente é instalado o Java Runtime Environment (JRE) versão 8.

In [0]:
#instalando Java Runtime Environment (JRE) versão 8
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Na sequência, é feito o *download* do Apache Spark versão 3.0.0.

In [0]:
#baixando Apache Spark versão 3.0.0
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz

Na sequência, são configuradas as variáveis de ambiente JAVA_HOME e SPARK_HOME. Isto permite que tanto o Java quanto o Spark possam ser encontrados.

In [0]:
import os
#configurando a variável de ambiente JAVA_HOME
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#configurando a variável de ambiente SPARK_HOME
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

Por fim, são instalados dois pacotes da linguagem de programação Python, cujas funcionalidades são descritas a seguir.

> **Pacote findspark:** Usado para ler a variável de ambiente SPARK_HOME e armazenar seu valor na variável dinâmica de ambiente PYTHONPATH. Como resultado, Python pode encontrar a instalação do Spark.

> **Pacote pyspark:** PySpark é a API do Python para Spark. Ela possibilita o uso de Python, considerando que o *framework* Apache Spark encontra-se desenvolvido na linguagem de programação Scala.

In [0]:
%%capture
#instalando o pacote findspark
!pip install -q findspark==1.4.2
#instalando o pacote pyspark
!pip install -q pyspark==3.0.0

## 3.2 Conexão

PySpark não é adicionado ao *sys.path* por padrão. Isso significa que não é possível importá-lo, pois o interpretador da linguagem Python não sabe onde encontrá-lo.

Para resolver esse aspecto, é necessário instalar o módulo `findspark`. Esse módulo mostra onde PySpark está localizado. Os comandos a seguir têm essa finalidade.


In [0]:
#importando o módulo findspark
import findspark
#carregando a variávels SPARK_HOME na variável dinâmica PYTHONPATH
findspark.init()

Depois de configurados os pacotes e módulos e inicializadas as variáveis de ambiente, é possível iniciar o uso do Spark na aplicação de `data warehousing`. Para tanto, é necessário importar o comando `SparkSession` do módulo `pyspark.sql`. São utilizados os seguintes conceitos: <br>

- `SparkSession`: permite a criação de `DataFrames`. Como resultado, as tabelas relacionais podem ser manipuladas por meio de `DataFrames` e é possível realizar consultas OLAP por meio de comandos SQL. <br>
- `builder`: cria uma instância de SparkSession. <br>
- `appName`: define um nome para a aplicação, o qual pode ser visto na interface de usuário web do Spark. <br>
- `master`: define onde está o nó mestre do *cluster*. Como a aplicação é executada localmente e não em um *cluster*, indica-se isso pela *string* `local` seguida do parâmetro `[*]`. Ou seja, define-se que apenas núcleos locais são utilizados.
- `getOrCreate`: cria uma SparkSession. Caso ela já exista, retorna a instância existente.


**Observação**: A lista completa de todos os parâmetros que podem ser utilizados na inicialização do *cluster* pode ser encontrada neste [link](https://spark.apache.org/docs/latest/spark-standalone.html#cluster-launch-scripts).

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()

# 4 Geração dos DataFrames em Pandas da BI Solutions

Nesta seção são gerados os DataFrames em Pandas. Atenção aos nomes desses DataFrames.


In [0]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [0]:
cargoPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cargo.csv')
clientePandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cliente.csv')
dataPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/data.csv')
equipePandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/equipe.csv')
funcionarioPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/funcionario.csv')
negociacaoPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/negociacao.csv')
pagamentoPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/pagamento.csv')

# 5 Geração dos DataFrames em Spark da BI Solutions

Nesta seção são gerados dos DataFrames em Spark. Atenção aos nomes desses DataFrames.


## 5.1 Criação dos DataFrames

In [0]:
#criando os DataFrames em Spark
cargo = spark.read.csv(path="data/cargo.csv", header=True, sep=",")
cliente = spark.read.csv(path="data/cliente.csv", header=True, sep=",")
data = spark.read.csv(path="data/data.csv", header=True, sep=",")
equipe = spark.read.csv(path="data/equipe.csv", header=True, sep=",")
funcionario = spark.read.csv(path="data/funcionario.csv", header=True, sep=",")
negociacao = spark.read.csv(path="data/negociacao.csv", header=True, sep=",")
pagamento = spark.read.csv(path="data/pagamento.csv", header=True, sep=",")

## 5.2 Atualização dos Tipos de Dados

Nos comandos a seguir, primeiro são identificados quais colunas de quais `DataFrames` devem ser do tipo de dado inteiro. Na sequência, ocorre a conversão. Por fim, são exibidos os esquemas dos `DataFrames`, possibilitando visualizar a mudança de tipo de dados das colunas especificadas.

In [0]:
# identificando quais colunas de quais DataFrames devem ser do tipo de dado inteiro
colunas_cargo = ["cargoPK"]
colunas_cliente = ["clientePK"]
colunas_data = ["dataPK", "dataDia", "dataMes", "dataBimestre", "dataTrimestre", "dataSemestre", "dataAno"]
colunas_equipe = ["equipePK"]
colunas_funcionario = ["funcPK", "funcDiaNascimento", "funcMesNascimento", "funcAnoNascimento"]
colunas_negociacao = ["equipePK", "clientePK", "dataPK", "quantidadeNegociacoes"]
colunas_pagamento = ["funcPK", "equipePK", "dataPK", "cargoPK", "quantidadeLancamentos"]

In [0]:
# importando o tipo de dado desejado
from pyspark.sql.types import IntegerType


# atualizando o tipo de dado das colunas especificadas
# substituindo as colunas já existentes

for coluna in colunas_cargo:
  cargo = cargo.withColumn(coluna, cargo[coluna].cast(IntegerType()))

for coluna in colunas_cliente:
  cliente = cliente.withColumn(coluna, cliente[coluna].cast(IntegerType()))

for coluna in colunas_data:
  data = data.withColumn(coluna, data[coluna].cast(IntegerType()))

for coluna in colunas_equipe:
  equipe = equipe.withColumn(coluna, equipe[coluna].cast(IntegerType()))

for coluna in colunas_funcionario:
  funcionario = funcionario.withColumn(coluna, funcionario[coluna].cast(IntegerType()))

for coluna in colunas_negociacao:
  negociacao = negociacao.withColumn(coluna, negociacao[coluna].cast(IntegerType()))

for coluna in colunas_pagamento:
  pagamento = pagamento.withColumn(coluna, pagamento[coluna].cast(IntegerType()))

Nos comandos a seguir, primeiro são identificados quais colunas de quais `DataFrames` devem ser do tipo de dado número de ponto flutuante. Na sequência, ocorre a conversão. Por fim, são exibidos os esquemas dos `DataFrames`, possibilitando visualizar a mudança de tipo de dados das colunas especificadas.

In [0]:
# identificando quais colunas de quais DataFrames devem ser do tipo de dado número de ponto flutuante
colunas_negociacao = ["receita"]
colunas_pagamento = ["salario"]

In [0]:
# importando o tipo de dado desejado
from pyspark.sql.types import FloatType


# atualizando o tipo de dado das colunas especificadas
# substituindo as colunas já existentes

for coluna in colunas_negociacao:
  negociacao = negociacao.withColumn(coluna, negociacao[coluna].cast(FloatType()))

for coluna in colunas_pagamento:
  pagamento = pagamento.withColumn(coluna, pagamento[coluna].cast(FloatType()))

In [0]:
# importando funções adicionais
from pyspark.sql.functions import round, desc

## 5.3 Criação de Visões Temporárias

In [0]:
#criando as visões temporárias
cargo.createOrReplaceTempView("cargo")
cliente.createOrReplaceTempView("cliente")
data.createOrReplaceTempView("data")
equipe.createOrReplaceTempView("equipe")
funcionario.createOrReplaceTempView("funcionario")
negociacao.createOrReplaceTempView("negociacao")
pagamento.createOrReplaceTempView("pagamento")

# Re - baixando os dados

Tive problemas para baixar os dados da forma como a professora baixou e tive que baixar desta maneira.

In [0]:
import pandas as pd
from pyspark.sql import functions as f
from pyspark.sql import SparkSession


In [0]:
# Criar a sessão Spark
spark = SparkSession.builder.appName("ExemploSQL").getOrCreate()

# Lista de URLs
lista_url = [
    "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/data.csv",
    "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/funcionario.csv",
    "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/equipe.csv",
    "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cargo.csv",
    "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cliente.csv",
    "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/pagamento.csv",
    "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/negociacao.csv"
]

data_pd = pd.read_csv(lista_url[0])
data = spark.createDataFrame(data_pd)

# Lista de DataFrames dataframes = [df1, df2] # Imprimir o schema de cada DataFrame for i, df in enumerate(dataframes, 1): print(f"Schema do DataFrame {i}:") df.printSchema() print("\n") = spark.createDataFrame(data_pd)

func_pd = pd.read_csv(lista_url[1])
funcionario = spark.createDataFrame(func_pd)

equipe_pd = pd.read_csv(lista_url[2])
equipe = spark.createDataFrame(equipe_pd)

cargo_pd = pd.read_csv(lista_url[3])
cargo = spark.createDataFrame(cargo_pd)

cliente_pd = pd.read_csv(lista_url[4])
cliente = spark.createDataFrame(cliente_pd)

pagamento_pd = pd.read_csv(lista_url[5])
pagamento = spark.createDataFrame(pagamento_pd)

negociacao_pd = pd.read_csv(lista_url[6])
negociacao = spark.createDataFrame(negociacao_pd)



In [0]:
#criando as visões temporárias
cargo.createOrReplaceTempView("cargo")
cliente.createOrReplaceTempView("cliente")
data.createOrReplaceTempView("data")
equipe.createOrReplaceTempView("equipe")
funcionario.createOrReplaceTempView("funcionario")
negociacao.createOrReplaceTempView("negociacao")
pagamento.createOrReplaceTempView("pagamento")

# 6 Respostas

## Questão 04

In [0]:
import pandas as pd

pessoas = {
    "cpf": ["204.927.060-77", "116.948.760-20", "327.639.610-61",
            "904.716.030-40", "750.286.140-83"],
    "nome": ["Joao Paulo", "Jose Carlos", "Maria Eduarda",
             "Ana Julia", "Carlos Alberto"],
    "idade": [21, 23, 20, 21, 22],
    "viagem": [True, True, True, True, False]
}

df_pessoas = pd.DataFrame(pessoas)

viagem1 = {
    "cpf": ["204.927.060-77", "116.948.760-20", "327.639.610-61", "904.716.030-40"],
    "preco_viagem_1": [900.0, 250.0, 150.0, 8000.0]
}

df1 = pd.DataFrame(viagem1)

viagem2 = {
    "cpf": ["750.286.140-83", "116.948.760-20", "327.639.610-61", "904.716.030-40"],
    "preco_viagem_2": [345.0, 4978.0, 8000.0, 9000.0]
}

df2 = pd.DataFrame(viagem2)

dfR = df_pessoas\
    .query("viagem == True")\
    .merge(df1, how="left", on="cpf")\
    .merge(df2, how="left", on="cpf")\
    .fillna(0)\
    .query("preco_viagem_1 + preco_viagem_2 < 1000")

dfR = dfR[["cpf", "nome", "preco_viagem_1", "preco_viagem_2"]]

dfR


,cpf,nome,preco_viagem_1,preco_viagem_2
0,204.927.060-77,Joao Paulo,900.0,0.0


## Questão 06

In [0]:
display(
  pagamento
.join(data, on="dataPK")
.join(cargo, on="cargoPK")
.join(funcionario, on="funcPK")
.where("dataAno >= 2017 and dataAno <= 2019")
.select("dataAno", "cargoNivel", "funcSexo", "salario")
.groupBy("dataAno", "cargoNivel", "funcSexo").avg("salario")
.orderBy("cargoNivel","funcSexo")
.withColumn("avg(salario)", f.round("avg(salario)",2))
.withColumnRenamed("avg(salario)", "Média dos Salários")
)

dataAno,cargoNivel,funcSexo,Média dos Salários
2017,JUNIOR,F,2793.01
2018,JUNIOR,F,2576.66
2019,JUNIOR,F,2440.23
2017,JUNIOR,M,2658.8
2018,JUNIOR,M,2509.45
2019,JUNIOR,M,2437.86
2017,PLENO,F,7236.78
2018,PLENO,F,7947.47
2019,PLENO,F,7641.94
2017,PLENO,M,6410.48


Lembre-se que é possível especificar as consultas analíticas usando Pandas, o método spark.sql() ou os métodos do módulo pyspark.sql.

## Questão 7

Considere a constelação de fatos da BI Solutions e a seguinte solicitação de consulta: “Liste todas as agregações que podem ser geradas a partir da média dos salários dos funcionários que moram na região NORDESTE por sexo e por ano de nascimento”. Arredonde a média dos salários para até duas casas decimais. Devem ser exibidas as colunas na ordem e com os nomes especificados a seguir: SEXO, ANO e MEDIASALARIO. Ordene as linhas exibidas primeiro por sexo, depois por ano de nascimento, depois por média dos salários, todos em ordem ascendente.

In [0]:
display(
  pagamento
  .join(funcionario, on='funcPK', how='full')
  .filter(f.col('funcRegiaoNome') == 'NORDESTE')
  .cube(['funcAnoNascimento', 'funcSexo'])
  .agg(f.round(f.mean('salario'), 2).alias('MEDIASALARIO'))
  .orderBy(['funcSexo', 'funcAnoNascimento', 'MEDIASALARIO'], ascending=True)
  .select(f.col('funcSexo').alias('SEXO'), f.col('funcAnoNascimento').alias('ANO'), 'MEDIASALARIO')
)

SEXO,ANO,MEDIASALARIO
null,null,7151.73
null,1951,4116.83
null,1952,3635.55
null,1953,11842.55
null,1965,1919.34
null,1966,14995.02
null,1967,6686.45
null,1978,2581.24
null,1979,6300.52
null,1980,2085.85


In [0]:
# Resposta da Questão 7
# Não se esqueça de comentar detalhadamente a sua solução

## Questão 8

In [0]:
display(
  cliente
  .join(negociacao, on='clientePK', how='inner')
  .join(data, on='dataPK', how='inner')
  .filter(
    ((f.col('dataAno') == 2019) | ((f.col('dataAno') == 2020)))
    & (f.col('clienteEstadoNome') == 'SAO PAULO')
    )
  .groupBy('clienteSetor', 'dataAno')
  .agg(f.round(f.sum('receita'), 2).alias('TOTALRECEITA'))
  .select(f.col("dataAno").alias('ANO'), f.col('clienteSetor').alias('SETOR'), 'TOTALRECEITA')
  .filter(f.col('TOTALRECEITA') > 3900000.00)
)

ANO,SETOR,TOTALRECEITA
2019,BEBIDAS E ALIMENTOS,3995904.25
2019,TECNOLOGIA,5778946.3
2020,BEBIDAS E ALIMENTOS,4264757.85


In [0]:
# Resposta da Questão 8
# Não se esqueça de comentar detalhadamente a sua solução



## Questão 9

In [0]:
display(
  equipe
  .join(negociacao, on='equipePK', how='inner')
  .join(data, on='dataPK', how='inner')
  .withColumn(
    "RECEITA2019", f.when(
       f.col('dataAno') == 2019, f.col('receita')).otherwise(0)
      )
  .withColumn(
    "RECEITA2020", f.when(
       f.col('dataAno') == 2020, f.col('receita')).otherwise(0)
      )
  .withColumn(
    "DIFERENCA", f.col('RECEITA2020') - f.col('RECEITA2019')
  )
  .filter(
    ((f.col('dataAno') == 2019) | ((f.col('dataAno') == 2020)))
    )
  .groupBy('equipeNome', 'filialNome')
  .agg( 
       f.round(f.sum("RECEITA2019"), 2).alias("RECEITA2019")
       , f.round(f.sum("RECEITA2020"), 2).alias("RECEITA2020")
       , f.round(f.sum("DIFERENCA")).alias("DIFERENCA") )
  .select(
    f.col('equipeNome').alias("EQUIPENOME")
    , f.col("filialNome").alias("EQUIPEFILIAL")
    , "RECEITA2019", "RECEITA2020", "DIFERENCA"
  )
 #.orderBy('RECEITA2019', ascending=False)
)

EQUIPENOME,EQUIPEFILIAL,RECEITA2019,RECEITA2020,DIFERENCA
BI & ANALYTICS,RECIFE - CENTRO,1.23106465E7,8791572.9,-3519074.0
BI & ANALYTICS,SAO PAULO - AV. PAULISTA,1.12672264E7,1.073077355E7,-536453.0
APP - DESKTOP,RIO DE JANEIRO - BARRA DA TIJUCA,2417619.9,2290441.3,-127179.0
APP - DESKTOP,SAO PAULO - AV. PAULISTA,2009714.0,2146181.25,136467.0
APP - MOBILE,SAO PAULO - AV. PAULISTA,1779690.6,1243670.55,-536020.0
APP - MOBILE,RIO DE JANEIRO - BARRA DA TIJUCA,1645371.0,1289305.0,-356066.0
APP - MOBILE,CAMPO GRANDE - CENTRO,1280914.9,1347929.7,67015.0
WEB,RIO DE JANEIRO - BARRA DA TIJUCA,1037993.7,612673.9,-425320.0
WEB,CAMPO GRANDE - CENTRO,956287.3,1017644.05,61357.0
WEB,SAO PAULO - AV. PAULISTA,647854.0,751983.75,104130.0


In [0]:
# Resposta da Questão 9
# Não se esqueça de comentar detalhadamente a sua solução



## Questão 10

In [0]:
display(
  funcionario
  .join(pagamento, on='funcPK', how='inner')
  .join(data, on='dataPK', how='inner')
  .join(cargo, on='cargoPK', how='inner')
  .filter(
    (f.col('dataAno') == 2020)
    & (f.col('cargoNivel') == 'SENIOR')
  )
  .groupBy('funcEstadoNome', 'funcRegiaoNome')
  .agg(f.round(f.sum('salario'), 2).alias('TOTALSALARIOS'))
  .select(
    f.col("funcEstadoNome").alias('ESTADO'), f.col('funcRegiaoNome').alias('REGIAO'), 'TOTALSALARIOS')
  .orderBy('TOTALSALARIOS', ascending=False)
)

ESTADO,REGIAO,TOTALSALARIOS
SAO PAULO,SUDESTE,4623213.72
MINAS GERAIS,SUDESTE,2133676.2
PARANA,SUL,2006090.28
RIO DE JANEIRO,SUDESTE,1094470.32
PERNAMBUCO,NORDESTE,822273.24


In [0]:
4623213.72-(2133676.2+1094470.32)

Out[87]: 1395067.1999999993

In [0]:
2133676.2-(2006090.28+822273.24)

Out[89]: -694687.3199999998

In [0]:
# Resposta da Questão 10
# Não se esqueça de comentar detalhadamente a sua solução
